In [14]:
## Import required packages
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model

## Create a list of workers and shifts
shiftList = ['Monday1','Monday2','Tuesday1','Tuesday2','Wednesday1','Wednesday2'
             ,'Thursday1','Thursday2','Friday1','Friday2','Saturday1','Saturday2','Sunday1','Sunday2']
workerList = ['EE01','EE02','EE03','EE04','EE05','EE06','EE07','EE08','EE09','EE10']

## Define shift requirements
shiftReq = [3,2,4,4,5,4,5,4,2,4,5,4,3,5]
shiftRequirements  = { s : shiftReq[i] for i,s in enumerate(shiftList) }

## Clarify worker availability
# Assume everyone is available
availability = pd.DataFrame(np.ones((len(workerList), len(shiftList))), index=workerList, columns=shiftList)
# For illustration, assume following people are unavailable: EE02 on Tuesday1, EE05 on Saturday2, EE08 on Thursday1
availability.at['EE02','Tuesday1'] = 0
availability.at['EE05','Saturday2'] = 0
availability.at['EE08','Thursday1'] = 0
# Create dictionary of final worker availability
avail = {(w,s) : availability.loc[w,s] for w in workerList for s in shiftList}

## Specify who is a manager and who isn't
mgmtList = ['EE01','EE03','EE05','EE07']
mgmtIndList = [0, 2, 4, 6]
nonmgmtList = [x for x in workerList if x not in mgmtList]

## Define total shift cost per worker
# Cost of a regular shift
regCost = [200,100,225,110,190,105,210,120,95,100]
# Cost of overtime shift
OTCost = [1.5*x for x in regCost]
# Create dictionaries with costs
regularCost  = { w : regCost[i] for i,w in enumerate(workerList) }
overtimeCost  = { w : OTCost[i] for i,w in enumerate(workerList) }

pref_matrix = np.array(
                       [[ 5., 14.,  8.,  2.,  3.,  7.,  4., 11.,  1.,  6., 13., 12., 10., 9.],
                        [ 1., 14.,  9.,  3., 12., 13., 11.,  4.,  7.,  2.,  8.,  6.,  5., 10.],
                        [ 7.,  1.,  3.,  2., 12., 13.,  6.,  9., 10.,  5.,  4., 14.,  8., 11.],
                        [ 6.,  9., 12., 13.,  3.,  5., 14., 10.,  2.,  4.,  7., 11.,  1., 8.],
                        [10., 12., 13.,  6.,  2., 11.,  9.,  8.,  4.,  1.,  7., 14.,  3., 5.],
                        [ 8., 13., 12.,  4., 14.,  7.,  6.,  1.,  2.,  3., 11.,  9.,  5., 10.],
                        [ 5.,  8., 10., 13.,  4.,  1., 11.,  6., 14.,  7.,  9., 12.,  3., 2.],
                        [ 9.,  5., 13., 14., 11., 10.,  7.,  1., 12.,  6.,  2.,  4.,  8., 3.],
                        [ 4.,  3., 10., 11.,  6.,  8.,  2., 12., 14.,  9.,  1.,  7.,  5., 13.],
                        [ 1.,  2.,  4.,  6.,  5.,  8.,  3.,  7., 11., 13., 10.,  9., 14., 12.]])

## Input assumptions
# Range of shifts that every workers is required to stay between
minShifts = 3
maxShifts = 7
# Number of shifts to trigger overtime
OTTrigger = 5

In [2]:
availability.iloc[1, 2]

0.0

In [3]:
solver = pywraplp.Solver('SolveShedulingtProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

x = {}
for s in range(len(shiftList)):
    for w in range(len(workerList)):
        x[w, s] = solver.IntVar(0, availability.iloc[w, s], 'x[%i,%i]' % (w, s))

In [4]:
reg_hours = {}
overtime_hours = {}

for w in range(len(workerList)):
    reg_hours[w] = solver.IntVar(minShifts, OTTrigger, 'reg_hrs%i' % w)
    overtime_hours[w] = solver.IntVar(0,  maxShifts - OTTrigger, 'ovrtime_hrs%i' % w)
    solver.Add(solver.Sum([x[w, s] for s in range(len(shiftList))]) == reg_hours[w] + overtime_hours[w])

In [5]:
for s in range(len(shiftList)):
    solver.Add(solver.Sum([x[w, s] for w in range(len(workerList))]) == shiftReq[s])
    solver.Add(solver.Sum([x[w, s] for w in mgmtIndList]) >= 1)

In [6]:
solver.Minimize(solver.Sum([reg_hours[w]*regCost[w] + 1.5*overtime_hours[w]*regCost[w] for w in range(len(workerList))]))

In [7]:
sol = solver.Solve()
sol

0

In [8]:
solver.Objective().Value()

7535.0

In [9]:
for i in range(len(workerList)):
    print (x[i,3].solution_value())

0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
1.0
1.0


In [10]:
availability

,Monday1,Monday2,Tuesday1,Tuesday2,Wednesday1,Wednesday2,Thursday1,Thursday2,Friday1,Friday2,Saturday1,Saturday2,Sunday1,Sunday2
EE01,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
EE02,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
EE03,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
EE04,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
EE05,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
EE06,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
EE07,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
EE08,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
EE09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
EE10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [11]:
reg_hours[2].solution_value()

3.0

In [12]:
workforce_solution = {workerList[w]: [x[w, s].solution_value() for s in range(len(shiftList))] for w in range(len(workerList))}

In [13]:
pd.DataFrame.from_dict(workforce_solution, orient='index', columns=shiftList)

,Monday1,Monday2,Tuesday1,Tuesday2,Wednesday1,Wednesday2,Thursday1,Thursday2,Friday1,Friday2,Saturday1,Saturday2,Sunday1,Sunday2
EE01,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
EE02,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
EE03,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
EE04,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
EE05,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
EE06,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
EE07,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
EE08,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
EE09,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
EE10,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0


In [9]:
model = cp_model.CpModel()

In [10]:
x = {}
for s in range(len(shiftList)):
    for w in range(len(workerList)):
        x[w, s] = model.NewIntVar(0, int(availability.iloc[w, s]), 'x[%i,%i]' % (w, s))

In [11]:
reg_hours = {}
overtime_hours = {}

for w in range(len(workerList)):
    reg_hours[w] = model.NewIntVar(minShifts, OTTrigger, 'reg_hrs%i' % w)
    overtime_hours[w] = model.NewIntVar(0, maxShifts - OTTrigger, 'ovrtime_hrs%i' % w)
    model.Add(sum(x[w, s] for s in range(len(shiftList))) == reg_hours[w] + overtime_hours[w])

In [12]:
for s in range(len(shiftList)):
    model.Add(sum([x[w, s] for w in range(len(workerList))]) == shiftReq[s])
    model.Add(sum([x[w, s] for w in mgmtIndList]) >= 1)

In [13]:
model.Minimize(sum(2*reg_hours[w]*regCost[w] + 3*overtime_hours[w]*regCost[w] for w in range(len(workerList))))

In [ ]:
solver = cp_model.CpSolver()
solver.Solve(model)

In [ ]:
solver.ObjectiveValue()/2

In [27]:
workforce_solution = {workerList[w]: [solver.Value(x[w, s]) for s in range(len(shiftList))] for w in range(len(workerList))}
pd.DataFrame.from_dict(workforce_solution, orient='index', columns=shiftList)

,Monday1,Monday2,Tuesday1,Tuesday2,Wednesday1,Wednesday2,Thursday1,Thursday2,Friday1,Friday2,Saturday1,Saturday2,Sunday1,Sunday2
EE01,0,0,0,0,1,0,1,0,0,0,0,0,0,1
EE02,0,1,0,1,1,0,1,1,0,0,1,0,1,0
EE03,0,0,0,1,1,0,0,0,0,0,0,0,0,1
EE04,0,1,1,0,0,0,1,1,0,1,1,1,0,0
EE05,0,0,1,0,0,1,0,0,0,1,0,0,0,0
EE06,0,0,1,0,0,1,0,1,1,1,1,1,0,0
EE07,1,0,0,0,0,0,0,0,0,0,1,0,0,1
EE08,0,0,1,0,0,0,0,1,1,1,1,1,1,0
EE09,1,0,0,1,1,1,1,0,0,0,0,1,0,1
EE10,1,0,0,1,1,1,1,0,0,0,0,0,1,1


In [9]:
from __future__ import print_function
from ortools.sat.python import cp_model


def main():
    # This program tries to find an optimal assignment of nurses to shifts
    # (3 shifts per day, for 7 days), subject to some constraints (see below).
    # Each nurse can request to be assigned to specific shifts.
    # The optimal assignment maximizes the number of fulfilled shift requests.
    num_nurses = 5
    num_shifts = 3
    num_days = 7
    all_nurses = range(num_nurses)
    all_shifts = range(num_shifts)
    all_days = range(num_days)
    shift_requests = [[[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1],
                       [0, 1, 0], [0, 0, 1]],
                      [[0, 0, 0], [0, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0],
                       [0, 0, 0], [0, 0, 1]],
                      [[0, 1, 0], [0, 1, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0],
                       [0, 1, 0], [0, 0, 0]],
                      [[0, 0, 1], [0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 0],
                       [1, 0, 0], [0, 0, 0]],
                      [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 0], [1, 0, 0],
                       [0, 1, 0], [0, 0, 0]]]
    # Creates the model.
    model = cp_model.CpModel()

    # Creates shift variables.
    # shifts[(n, d, s)]: nurse 'n' works shift 's' on day 'd'.
    shifts = {}
    for n in all_nurses:
        for d in all_days:
            for s in all_shifts:
                shifts[(n, d,
                        s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

    # Each shift is assigned to exactly one nurse in .
    for d in all_days:
        for s in all_shifts:
            model.Add(sum(shifts[(n, d, s)] for n in all_nurses) == 1)

    # Each nurse works at most one shift per day.
    for n in all_nurses:
        for d in all_days:
            model.Add(sum(shifts[(n, d, s)] for s in all_shifts) <= 1)

    # min_shifts_assigned is the largest integer such that every nurse can be
    # assigned at least that number of shifts.
    min_shifts_per_nurse = (num_shifts * num_days) // num_nurses
    max_shifts_per_nurse = min_shifts_per_nurse + 1
    for n in all_nurses:
        num_shifts_worked = sum(
            shifts[(n, d, s)] for d in all_days for s in all_shifts)
        model.Add(min_shifts_per_nurse <= num_shifts_worked)
        model.Add(num_shifts_worked <= max_shifts_per_nurse)

    model.Maximize(
        sum(shift_requests[n][d][s] * shifts[(n, d, s)] for n in all_nurses
            for d in all_days for s in all_shifts))
    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    solver.Solve(model)
    for d in all_days:
        print('Day', d)
        for n in all_nurses:
            for s in all_shifts:
                if solver.Value(shifts[(n, d, s)]) == 1:
                    if shift_requests[n][d][s] == 1:
                        print('Nurse', n, 'works shift', s, '(requested).')
                    else:
                        print('Nurse', n, 'works shift', s, '(not requested).')
        print()

    # Statistics.
    print()
    print('Statistics')
    print('  - Number of shift requests met = %i' % solver.ObjectiveValue(),
          '(out of', num_nurses * min_shifts_per_nurse, ')')
    print('  - wall time       : %f s' % solver.WallTime())


if __name__ == '__main__':
    main()

Day 0
Nurse 1 works shift 0 (not requested).
Nurse 2 works shift 1 (requested).
Nurse 3 works shift 2 (requested).

Day 1
Nurse 0 works shift 0 (not requested).
Nurse 2 works shift 1 (requested).
Nurse 4 works shift 2 (requested).

Day 2
Nurse 1 works shift 1 (requested).
Nurse 3 works shift 0 (requested).
Nurse 4 works shift 2 (not requested).

Day 3
Nurse 2 works shift 0 (requested).
Nurse 3 works shift 1 (requested).
Nurse 4 works shift 2 (not requested).

Day 4
Nurse 0 works shift 2 (requested).
Nurse 1 works shift 0 (requested).
Nurse 4 works shift 1 (not requested).

Day 5
Nurse 0 works shift 2 (not requested).
Nurse 2 works shift 1 (requested).
Nurse 3 works shift 0 (requested).

Day 6
Nurse 0 works shift 1 (not requested).
Nurse 1 works shift 2 (requested).
Nurse 2 works shift 0 (not requested).


Statistics
  - Number of shift requests met = 13 (out of 20 )
  - wall time       : 0.015198 s


# Optimize vessel loading

In [60]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model

## Create a list of workers and shifts
vesselList = ['V01','V02','V03','V4']
berthList = ['B01', 'B02']
#shipmentList = ['shipment01', 'shipment02', 'shipment03', 'shipment04', 'shipment05', 'shipment06', 'shipment07', 'shipment08']

vesselCap = [100, 150, 130, 120]

#shipmentWt = [20,10,25,11,19,15,21,12,20,10,25,11,19,15,21,12,20,10,25,11,19,15,21,12,20,10,25,11,19,15,21,12,20,10,25,11,19,15,21,12,20,10,25,11,19,15,21,12]
# shipmentWt = [15 for x in range(40)]
shipmentWt = [50,10,10,40]
vesselCost = [1000, 1200, 1100, 1100]

# MIP solver

In [47]:
solver = pywraplp.Solver('SolveShedulingtProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [48]:
x = {}
for v in range(len(vesselList)):
    for s in range(len(shipmentWt)):
        x[v, s] = solver.IntVar(0, 1, 'x[%i,%i]' % (v, s))

In [49]:
vessel_flag = {}
for v in range(len(vesselList)):
    vessel_flag[v] = solver.IntVar(0, 1, 'vessel_%i_filled' % (v))
    solver.Add(solver.Sum([shipmentWt[s] * x[v, s] for s in range(len(shipmentWt))]) <= vesselCap[v])
    [solver.Add(vessel_flag[v] >= x[v, s]) for s in range(len(shipmentWt))]

In [50]:
for s in range(len(shipmentWt)):
    solver.Add(solver.Sum([x[v, s] for v in range(len(vesselList))]) <= 1)

In [51]:
solver.Maximize(solver.Sum([shipmentWt[s] * x[v, s] for v in range(len(vesselList)) for s in range(len(shipmentWt))]) 
               - min(shipmentWt) * solver.Sum([vessel_flag[k] for k in range(len(vessel_flag))]))

In [52]:
solver.Solve()

0

In [53]:
solver.Objective().Value()

460.0

In [54]:
print ('Total tonnes filled', sum([shipmentWt[s] * x[v, s].solution_value() for v in range(len(vesselList)) for s in range(len(shipmentWt))]))

Total tonnes filled 500.0


In [55]:
solution = {vesselList[v]: [x[v, s].solution_value() for s in range(len(shipmentWt))] for v in range(len(vesselList))}

In [56]:
pd.DataFrame(solution, index=shipmentWt)

,V01,V02,V03,V4
20,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0
25,0.0,1.0,0.0,0.0
11,0.0,0.0,0.0,0.0
19,0.0,0.0,0.0,1.0
15,0.0,1.0,0.0,0.0
21,0.0,0.0,0.0,0.0
12,0.0,1.0,0.0,0.0
20,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,1.0


In [57]:
sum(shipmentWt), sum(vesselCap)

(798, 500)

In [58]:
print ('total boxes transferred', sum(map(sum, solution.values())))

total boxes transferred 30.0


In [59]:
print ([vessel_flag[k].solution_value() for k in range(len(vessel_flag))])

[1.0, 1.0, 1.0, 1.0]


# CP_SAT solver

In [72]:
model = cp_model.CpModel()

In [73]:
x = {}
for v in range(len(vesselList)):
    for s in range(len(shipmentWt)):
        x[v, s] = model.NewIntVar(0, 1, 'x[%i,%i]' % (v, s))

In [74]:
vessel_flag = {}
for v in range(len(vesselList)):
    vessel_flag[v] = model.NewIntVar(0, 1, 'vessel_%i_filled' % (v))
    model.Add(sum([shipmentWt[s] * x[v, s] for s in range(len(shipmentWt))]) <= vesselCap[v])
    [model.Add(vessel_flag[v] >= x[v, s]) for s in range(len(shipmentWt))]

In [75]:
for s in range(len(shipmentWt)):
    model.Add(sum([x[v, s] for v in range(len(vesselList))]) <= 1)

In [76]:
model.Maximize(sum([shipmentWt[s] * x[v, s] for v in range(len(vesselList)) for s in range(len(shipmentWt))]))

In [77]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [78]:
if status == cp_model.OPTIMAL:
    print('Maximum of objective function: %i' % solver.ObjectiveValue())

Maximum of objective function: 500


In [79]:
solution = {vesselList[v]: [solver.Value(x[v, s]) for s in range(len(shipmentWt))] for v in range(len(vesselList))}

In [83]:
pd.DataFrame(solution, index=shipmentWt)

,V01,V02,V03,V4
20,0,0,0,0
10,0,0,1,0
25,0,0,1,0
11,0,1,0,0
19,0,0,0,0
15,0,0,0,0
21,0,0,0,0
12,0,1,0,0
20,0,0,0,0
10,0,0,0,0


In [81]:
sum(shipmentWt), sum(vesselCap)

(798, 500)

In [82]:
print ('total boxes transferred', sum(map(sum, solution.values())))

total boxes transferred 31


# Cost as objective function

In [7]:
solver.Minimize(solver.Sum([vessel_flag[v] * vesselCost[v] for v in range(len(vesselList))]))

In [8]:
solver.Solve()

0

In [9]:
solver.Objective().Value()

0.0

In [12]:
mat = np.zeros((10,14))
a = np.arange(1,15)

for i in range(10):
    np.random.shuffle(a)
    mat[i] = a


In [2]:
import numpy as np

In [13]:
mat

array([[ 5., 14.,  8.,  2.,  3.,  7.,  4., 11.,  1.,  6., 13., 12., 10.,
         9.],
       [ 1., 14.,  9.,  3., 12., 13., 11.,  4.,  7.,  2.,  8.,  6.,  5.,
        10.],
       [ 7.,  1.,  3.,  2., 12., 13.,  6.,  9., 10.,  5.,  4., 14.,  8.,
        11.],
       [ 6.,  9., 12., 13.,  3.,  5., 14., 10.,  2.,  4.,  7., 11.,  1.,
         8.],
       [10., 12., 13.,  6.,  2., 11.,  9.,  8.,  4.,  1.,  7., 14.,  3.,
         5.],
       [ 8., 13., 12.,  4., 14.,  7.,  6.,  1.,  2.,  3., 11.,  9.,  5.,
        10.],
       [ 5.,  8., 10., 13.,  4.,  1., 11.,  6., 14.,  7.,  9., 12.,  3.,
         2.],
       [ 9.,  5., 13., 14., 11., 10.,  7.,  1., 12.,  6.,  2.,  4.,  8.,
         3.],
       [ 4.,  3., 10., 11.,  6.,  8.,  2., 12., 14.,  9.,  1.,  7.,  5.,
        13.],
       [ 1.,  2.,  4.,  6.,  5.,  8.,  3.,  7., 11., 13., 10.,  9., 14.,
        12.]])

In [ ]:
[[ 5., 14.,  8.,  2.,  3.,  7.,  4., 11.,  1.,  6., 13., 12., 10., 9.],
[ 1., 14.,  9.,  3., 12., 13., 11.,  4.,  7.,  2.,  8.,  6.,  5., 10.],
[ 7.,  1.,  3.,  2., 12., 13.,  6.,  9., 10.,  5.,  4., 14.,  8., 11.],
[ 6.,  9., 12., 13.,  3.,  5., 14., 10.,  2.,  4.,  7., 11.,  1., 8.],
[10., 12., 13.,  6.,  2., 11.,  9.,  8.,  4.,  1.,  7., 14.,  3., 5.],
[ 8., 13., 12.,  4., 14.,  7.,  6.,  1.,  2.,  3., 11.,  9.,  5., 10.],
[ 5.,  8., 10., 13.,  4.,  1., 11.,  6., 14.,  7.,  9., 12.,  3., 2.],
[ 9.,  5., 13., 14., 11., 10.,  7.,  1., 12.,  6.,  2.,  4.,  8., 3.],
[ 4.,  3., 10., 11.,  6.,  8.,  2., 12., 14.,  9.,  1.,  7.,  5., 13.],
[ 1.,  2.,  4.,  6.,  5.,  8.,  3.,  7., 11., 13., 10.,  9., 14., 12.]]

# Delivery Scheduling

### Using (vehicle, node, time step)

In [8]:

data = {}
data['distance_matrix'] = [
    [
        0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354,
        468, 776, 662
    ],
    [
        548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,
        1016, 868, 1210
    ],
    [
        776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,
        1130, 788, 1552, 754
    ],
    [
        696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,
        1164, 560, 1358
    ],
    [
        582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,
        1050, 674, 1244
    ],
    [
        274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628,
        514, 1050, 708
    ],
    [
        502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856,
        514, 1278, 480
    ],
    [
        194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320,
        662, 742, 856
    ],
    [
        308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662,
        320, 1084, 514
    ],
    [
        194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388,
        274, 810, 468
    ],
    [
        536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764,
        730, 388, 1152, 354
    ],
    [
        502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114,
        308, 650, 274, 844
    ],
    [
        388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194,
        536, 388, 730
    ],
    [
        354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0,
        342, 422, 536
    ],
    [
        468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536,
        342, 0, 764, 194
    ],
    [
        776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274,
        388, 422, 764, 0, 798
    ],
    [
        662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730,
        536, 194, 798, 0
    ],
]
data['num_locations'] = 17
data['num_vehicles'] = 4
data['depot'] = 0
data['max_deliveries'] = 5


In [9]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model

In [26]:
solver = pywraplp.Solver('SolveShedulingtProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [6]:
x = {}
for n in range(data['num_locations']):
    for v in range(data['num_vehicles']):
        for t in range(data['max_deliveries']):
            x[v, n, t] = solver.IntVar(0, 1, 'x[%i,%i,%i]' % (v, n, t))

In [7]:
solver.Add(solver.Sum([x[v,0,0] for v in range(data['num_vehicles'])]) == data['num_vehicles'])

for n in range(1, data['num_locations']):
    solver.Add(solver.Sum([x[v, n, t] for v in range(data['num_vehicles']) for t in range(data['max_deliveries'])]) == 1)

In [35]:
for v in range(data['num_vehicles']):
    for t in range(data['max_deliveries']):
        solver.Add(solver.Sum([x[v, n, t] for n in range(data['num_locations'])]) == 1)

In [13]:
for v in range(data['num_vehicles']):
    solver.Add(solver.Sum([x[v, n, t] for n in range(data['num_locations']) for t in range(data['max_deliveries'])]) <= data['max_deliveries'])

In [18]:
pos = {}
for v in range(data['num_vehicles']):
    for t in range(data['max_deliveries']):
        pos[v, t] = solver.Sum([n * x[v, n, t] for n in range(data['num_locations'])])

In [21]:
dist_travelled = {}
for v in range(data['num_vehicles']):
    dist_travelled[v] = solver.Sum([data['distance_matrix'][pos[v, t]][pos[v, (t-1)]] for t in range(1, data['max_deliveries'])])

TypeError: list indices must be integers or slices, not SumArray

In [27]:
x = {}
t = {}
for i in range(data['num_locations']):
    t[i] = solver.IntVar(0, 3, 't[%i]' % i)
    
for i in range(data['num_locations']):
    for j in range(data['num_locations']):
        x[i, j] = solver.IntVar(0, 1, 'x[%i,%i]' % (i, j))
        if i == j:
            solver.Add(x[i, j] == 0)
        if i != 0 and j != 0:
            solver.Add(t[i] - t[j] + data['num_locations'] * x[i, j] <= data['num_locations'] - 1)

In [28]:

for i in range(1, data['num_locations']):
    if i != j:
        solver.Add(solver.Sum([x[i, j] for j in range(data['num_locations'])]) == 1)
for j in range(1, data['num_locations']):
    if i != j:
        solver.Add(solver.Sum([x[i, j] for i in range(data['num_locations'])]) == 1)

solver.Add(solver.Sum([x[0, j] for j in range(data['num_locations'])]) == data['num_vehicles'])

solver.Add(solver.Sum([x[i, 0] for i in range(data['num_locations'])]) == data['num_vehicles'])

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x0000025F85448240> >

In [29]:
solver.Minimize(solver.Sum([data['distance_matrix'][i][j] * x[i, j] for i in range(data['num_locations']) for j in range(data['num_locations'])]))

In [30]:
solver.Solve()

0

In [31]:
solver.Objective().Value()

5912.0

In [32]:
solution = {j: [x[i, j].solution_value() for i in range(data['num_locations'])] for j in range(data['num_locations'])}

In [33]:
pd.DataFrame(solution)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
solver.OPTIMAL

0